In [1]:
!nvidia-smi

Sun Dec 29 21:20:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   78C    P3              11W /  35W |    431MiB /  4096MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/home/hasnan/Downloads/Notebook/HelmetPlusLicense


In [5]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.34 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3904MiB)
Setup complete ✅ (12 CPUs, 15.3 GB RAM, 187.2/361.0 GB disk)


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
import easyocr
from PIL import Image, ImageDraw, ImageFont

In [13]:
licence_model = YOLO('/home/hasnan/Downloads/Notebook/runs/detect/train8/weights/best.pt')
helmet_model = YOLO('/home/hasnan/Downloads/Notebook/Licence/runs/detect/train3/weights/best.pt')

In [29]:
#test print

import easyocr
reader = easyocr.Reader(['en'])
test_image = '/home/hasnan/Downloads/Notebook/test_1.jpg'
easyocr_results = reader.readtext(test_image)
for (bbox, text, confidence) in easyocr_results:
    print(f"Text: {text}, Confidence: {confidence:.2f}")

In [195]:
from ultralytics import YOLO
import easyocr
import cv2

#
# Initialize models
licence_model = YOLO('/home/hasnan/Downloads/Notebook/runs/detect/train8/weights/best.pt')
helmet_model = YOLO('/home/hasnan/Downloads/Notebook/Licence/runs/detect/train3/weights/best.pt')

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])  # Use 'bn' for Bengali if applicable

# List to store detected violations
violations = []
detected_plates = set()  # To store unique license plates


def process_image(image_path):
    global violations, detected_plates

    # Load the image
    image = cv2.imread(image_path)

    # Detect objects using the helmet model
    detections = helmet_model(image, conf=0.25)

    # Separate detections into persons and helmets
    person_boxes = [box for box in detections[0].boxes if box.cls == 1]  # Adjust class index for persons
    helmet_boxes = [box for box in detections[0].boxes if box.cls == 0]  # Adjust class index for helmets

    # Check if a helmet is worn by any detected person
    if not person_boxes and helmet_boxes:
        print("Violation detected: No helmet.")
        reader = easyocr.Reader(['en'])
        easyocr_results = reader.readtext(image_path)
        for (bbox, text, confidence) in easyocr_results:
            violations.append(text)
    else:
        print("No violations: Helmet detected.")


# Example usage
image_path = '/home/hasnan/Downloads/Notebook/steptodown.com873783.jpg'  # Replace with your test image path
process_image(image_path)

print("Violations:", violations)


0: 640x448 (no detections), 6.2ms
Speed: 1.5ms preprocess, 6.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 448)
No violations: Helmet detected.
Violations: []


In [171]:
from ultralytics import YOLO
import easyocr
import cv2

# Initialize models
helmet_model = YOLO('/home/hasnan/Downloads/Notebook/runs/detect/train8/weights/best.pt')
licence_model = YOLO('/home/hasnan/Downloads/Notebook/Licence/runs/detect/train3/weights/best.pt')

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])  # Use 'bn' for Bengali if applicable

# List to store detected violations
violations = []
detected_plates = set()  # To store unique license plates


def process_image(image_path):
    global violations, detected_plates

    # Load the image
    image = cv2.imread(image_path)

    # Detect objects using the helmet model
    detections = helmet_model(image, conf=0.25)

    # Separate detections into persons, helmets, and bikes
    bike_boxes = [box for box in detections[0].boxes if box.cls == 2]  # Adjust class index for bikes
    person_boxes = [box for box in detections[0].boxes if box.cls == 1]  # Adjust class index for persons
    helmet_boxes = [box for box in detections[0].boxes if box.cls == 0]  # Adjust class index for helmets

    for bike in bike_boxes:
        bx1, by1, bx2, by2 = map(int, bike.xyxy[0])
        nearby_persons = [
            person for person in person_boxes if calculate_overlap(bike, person) > 0.3
        ]

        if nearby_persons and not helmet_boxes:
            print("Violation detected: No helmet for a bike rider.")
            reader = easyocr.Reader(['en'])
            easyocr_results = reader.readtext(image_path)

            for (bbox, text, confidence) in easyocr_results:
                print(f"Text: {text}, Confidence: {confidence:.2f}")

                # Avoid duplicate entries
                if text not in detected_plates:
                    detected_plates.add(text)
                    violations.append(text)
        else:
            print("No violations: Helmet detected or no bike nearby.")


def calculate_overlap(box1, box2):
    """
    Calculate the overlap ratio between two bounding boxes.
    :param box1: YOLO box 1
    :param box2: YOLO box 2
    :return: Overlap ratio (IoU)
    """
    x1 = max(box1.xyxy[0][0], box2.xyxy[0][0])
    y1 = max(box1.xyxy[0][1], box2.xyxy[0][1])
    x2 = min(box1.xyxy[0][2], box2.xyxy[0][2])
    y2 = min(box1.xyxy[0][3], box2.xyxy[0][3])

    # Compute intersection area
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)

    # Compute areas of both boxes
    box1_area = (box1.xyxy[0][2] - box1.xyxy[0][0]) * (box1.xyxy[0][3] - box1.xyxy[0][1])
    box2_area = (box2.xyxy[0][2] - box2.xyxy[0][0]) * (box2.xyxy[0][3] - box2.xyxy[0][1])

    # Compute IoU
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou


# Example usage
#image_path = '/home/hasnan/Downloads/Notebook/steptodown.com873783.jpg'
image_path = '/home/hasnan/Downloads/Notebook/helmet-detection-18/train/images/28_jpg.rf.b444b6d15f1f62e74169c3f22d3b34c0.jpg'
process_image(image_path)

print("Violations:", violations)


0: 640x640 1 driver, 1 no-helmet, 10.6ms
Speed: 1.3ms preprocess, 10.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Violations: []


In [181]:
from ultralytics import YOLO
import easyocr
import cv2

helmet_model = YOLO('/home/hasnan/Downloads/Notebook/runs/detect/train8/weights/best.pt')
licence_model = YOLO('/home/hasnan/Downloads/Notebook/Licence/runs/detect/train3/weights/best.pt')

reader = easyocr.Reader(['en'])

violations = []
detected_plates = set()

def process_image(image_path):
    global violations, detected_plates

    image = cv2.imread(image_path)

    detections = helmet_model(image, conf=0.25)

    bike_boxes = [box for box in detections[0].boxes if box.cls == 2]
    person_boxes = [box for box in detections[0].boxes if box.cls == 1]
    helmet_boxes = [box for box in detections[0].boxes if box.cls == 0]

    for bike in bike_boxes:
        bx1, by1, bx2, by2 = map(int, bike.xyxy[0])
        nearby_persons = [
            person for person in person_boxes if calculate_overlap(bike, person) > 0.3
        ]

        if nearby_persons:
            rider = nearby_persons[0]

            helmet_near_rider = False
            rider_head_box = get_head_box(rider)

            for helmet in helmet_boxes:
                if calculate_overlap(rider_head_box, helmet) > 0.3:
                    helmet_near_rider = True
                    break

            if not helmet_near_rider:
                easyocr_results = reader.readtext(image_path)

                for (bbox, text, confidence) in easyocr_results:
                    if text not in detected_plates:
                        detected_plates.add(text)
                        violations.append(text)
            else:
                print("No violations: Helmet detected near rider's head.")
        else:
            print("No violations: No person detected riding the bike.")

def calculate_overlap(box1, box2):
    x1 = max(box1.xyxy[0][0], box2.xyxy[0][0])
    y1 = max(box1.xyxy[0][1], box2.xyxy[0][1])
    x2 = min(box1.xyxy[0][2], box2.xyxy[0][2])
    y2 = min(box1.xyxy[0][3], box2.xyxy[0][3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)

    box1_area = (box1.xyxy[0][2] - box1.xyxy[0][0]) * (box1.xyxy[0][3] - box1.xyxy[0][1])
    box2_area = (box2.xyxy[0][2] - box2.xyxy[0][0]) * (box2.xyxy[0][3] - box2.xyxy[0][1])

    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

def get_head_box(person_box):
    px1, py1, px2, py2 = map(int, person_box.xyxy[0])
    head_height = (py2 - py1) // 3  # Adjusted to estimate the head area as one-third of the person's bounding box height
    return [px1, py1, px2, py1 + head_height]

image_path = '/home/hasnan/Downloads/Notebook/steptodown.com873783.jpg'
process_image(image_path)

print("Violations:", violations)


0: 640x448 1 driver, 1 helmet, 7.7ms
Speed: 1.5ms preprocess, 7.7ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 448)
No violations: No person detected riding the bike.
Violations: []
